## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = glob.glob('/kaggle/input/117-googleq-a-train-bert-base-uncased-3f-lrwarmup2/' + '*.h5')
model_path_list.sort()
print('Models to predict:', model_path_list)

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict: ['/kaggle/input/117-googleq-a-train-bert-base-uncased-3f-lrwarmup2/model_fold_1.h5', '/kaggle/input/117-googleq-a-train-bert-base-uncased-3f-lrwarmup2/model_fold_2.h5', '/kaggle/input/117-googleq-a-train-bert-base-uncased-3f-lrwarmup2/model_fold_3.h5']
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map misspellings
    test[feature] = test[feature].apply(lambda x: map_misspellings(x))
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.950712,0.659427,0.191615,0.449730,0.602806,0.583434,0.711087,0.658635,0.598285,...,0.900194,0.891835,0.538013,0.953131,0.944133,0.785203,0.058815,0.038456,0.880416,0.910943
1,46,0.890245,0.511355,0.009191,0.756857,0.732032,0.906965,0.548947,0.467961,0.062982,...,0.679973,0.944273,0.664500,0.962705,0.976496,0.855559,0.921743,0.111325,0.077326,0.873709
2,70,0.922763,0.689231,0.033581,0.780252,0.856685,0.885115,0.602630,0.492624,0.143862,...,0.874927,0.925288,0.652655,0.958687,0.964970,0.860792,0.110088,0.059941,0.827899,0.903794
3,132,0.864182,0.376877,0.009033,0.678798,0.732351,0.897814,0.572200,0.437924,0.117982,...,0.662801,0.939376,0.658562,0.965703,0.980348,0.872288,0.888077,0.166312,0.352315,0.895584
4,200,0.918123,0.375035,0.061094,0.782318,0.678190,0.788448,0.683433,0.555732,0.084887,...,0.670664,0.873835,0.598545,0.953550,0.956432,0.793462,0.183423,0.105135,0.676824,0.903143


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.889332,0.579415,0.035063,0.705271,0.793998,0.842507,0.593914,0.481582,0.241266,...,0.775172,0.923009,0.655414,0.956311,0.966058,0.849424,0.527477,0.131824,0.507080,0.901992
std,2812.670060,0.050046,0.146292,0.058588,0.114628,0.103031,0.128410,0.053733,0.091628,0.208354,...,0.097119,0.026225,0.051954,0.013226,0.012986,0.043410,0.330785,0.065129,0.287429,0.021846
min,39.000000,0.734289,0.294188,0.003373,0.205321,0.313076,0.254978,0.483260,0.322716,0.006087,...,0.545054,0.799321,0.525106,0.898970,0.898690,0.712446,0.007810,0.006679,0.019772,0.828154
25%,2572.000000,0.853912,0.446265,0.009600,0.639614,0.743651,0.818656,0.555782,0.416360,0.086738,...,0.690377,0.910512,0.618139,0.948426,0.959654,0.821017,0.182980,0.081328,0.253355,0.887291
50%,5093.000000,0.893150,0.571192,0.015014,0.710098,0.797271,0.888268,0.582347,0.449691,0.162525,...,0.782345,0.926526,0.654958,0.957764,0.968419,0.855386,0.617757,0.132314,0.504996,0.904710
75%,7482.000000,0.929783,0.701536,0.027570,0.782351,0.863946,0.921059,0.627699,0.527573,0.338585,...,0.861750,0.941586,0.689160,0.966131,0.975664,0.881839,0.831965,0.176125,0.754836,0.917867
max,9640.000000,0.976986,0.896226,0.573393,0.964313,0.975423,0.977686,0.755934,0.763947,0.837691,...,0.955143,0.974593,0.818154,0.982306,0.990575,0.952801,0.974235,0.310957,0.984799,0.954135
